# VDEH Data Fusion Pipeline

**Fokus:** KI-gestützte Fusion von VDEH und DNB Daten

## 🎯 Ziel
- Intelligente Fusion von VDEH-Original und DNB-Daten
- Konfliktauflösung via Ollama LLM
- Vollständige Nachvollziehbarkeit aller Entscheidungen
- Qualitätsverbesserung durch Datenanreicherung

## 📚 Input/Output
- **Input**: `data/vdeh/processed/04_dnb_enriched_data.parquet`
- **Output**: `data/vdeh/processed/05_fused_data.parquet`

## 🤖 KI-Modell
- **Ollama**: Lokales LLM (llama3.2)
- **API**: http://localhost:11434

## 🔄 Fusion-Architektur


**Drei Fusion-Strategien:**
1. **Keine DNB-Daten** → VDEH behalten
2. **Keine Konflikte** → Einfacher Merge (VDEH priorisiert, DNB ergänzt)
3. **Konflikte vorhanden** → KI-Entscheidung via Ollama

**Vollständige Nachvollziehbarkeit:**
- `fusion_*_source`: Welche Quelle für jedes Feld
- `fusion_conflicts`: JSON mit allen erkannten Konflikten
- `fusion_ai_reasoning`: KI-Begründung der Entscheidung

In [1]:
# 🛠️ SETUP UND DATEN LADEN
import sys
import pandas as pd
import numpy as np
from pathlib import Path
import json
import warnings
import time
import requests
from typing import Dict, Optional, List

# Projektroot finden
current_dir = Path.cwd()
project_root = None

for parent in [current_dir] + list(current_dir.parents):
    if (parent / 'config.yaml').exists():
        project_root = parent
        break

if project_root is None:
    raise FileNotFoundError("config.yaml nicht gefunden!")

# Config laden
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from config_loader import load_config

config = load_config(project_root / 'config.yaml')

if not config.get('debug.verbose_output', True):
    warnings.filterwarnings('ignore')

print(f"📁 Projektroot: {project_root}")
print("✅ Konfiguration geladen")

✅ Konfiguration geladen: /media/sz/Data/Bibo/analysis/config.yaml
📁 Projektroot: /media/sz/Data/Bibo/analysis
✅ Konfiguration geladen


In [2]:
# 📂 DNB-ANGEREICHERTE DATEN LADEN
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')
input_path = processed_dir / '04_dnb_enriched_data.parquet'
metadata_path = processed_dir / '04_metadata.json'

if not input_path.exists():
    raise FileNotFoundError(f"Input-Datei nicht gefunden: {input_path}\n"
                          "Bitte führen Sie zuerst 04_vdeh_data_enrichment.ipynb aus.")

# Daten laden
df_enriched = pd.read_parquet(input_path)

# 🔧 CRITICAL: Konvertiere Categorical-Spalten zu String (falls vorhanden)
# Diese können von älteren Versionen der Input-Datei stammen
categorical_cols = df_enriched.select_dtypes(include=['category']).columns
if len(categorical_cols) > 0:
    print(f"⚠️  Konvertiere {len(categorical_cols)} Categorical-Spalten zu String:")
    for col in categorical_cols:
        df_enriched[col] = df_enriched[col].astype(str)
        print(f"   - {col}")
    print()

# Vorherige Metadaten laden
with open(metadata_path, 'r') as f:
    prev_metadata = json.load(f)

print(f"📂 Daten geladen aus: {input_path}")
print(f"📊 Records: {len(df_enriched):,}")
print(f"📋 Spalten: {list(df_enriched.columns)}")
print(f"💾 Memory: {df_enriched.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# DNB-Daten Statistiken
if 'dnb_query_method' in df_enriched.columns:
    dnb_records = df_enriched['dnb_query_method'].notna().sum()
    print(f"\n📊 DNB-Daten vorhanden: {dnb_records:,} ({dnb_records/len(df_enriched)*100:.1f}%)")
    
    method_counts = df_enriched['dnb_query_method'].value_counts()
    for method, count in method_counts.items():
        print(f"   {method}: {count:,}")

📂 Daten geladen aus: /media/sz/Data/Bibo/analysis/data/vdeh/processed/04_dnb_enriched_data.parquet
📊 Records: 58,760
📋 Spalten: ['id', 'title', 'authors', 'authors_affiliation', 'year', 'publisher', 'isbn', 'issn', 'authors_str', 'num_authors', 'authors_affiliation_str', 'num_authors_affiliation', 'isbn_valid', 'isbn_status', 'issn_valid', 'issn_status', 'lang_code', 'lang_confidence', 'lang_name', 'dnb_query_method', 'dnb_title', 'dnb_authors', 'dnb_year', 'dnb_publisher', 'dnb_title_ta', 'dnb_authors_ta', 'dnb_year_ta', 'dnb_publisher_ta']
💾 Memory: 63.1 MB

📊 DNB-Daten vorhanden: 6,232 (10.6%)
   ISBN: 6,232
💾 Memory: 63.1 MB

📊 DNB-Daten vorhanden: 6,232 (10.6%)
   ISBN: 6,232


In [3]:
# 📋 FUSION-SETUP
print("📋 === FUSION-SETUP ===\n")

# Ollama API-Konfiguration für Plausibilitätsprüfung
OLLAMA_API = "http://localhost:11434/api/generate"
OLLAMA_MODEL = "llama3.3:70b"  # Großes Modell für robustes Reasoning

# Konfigurierbare Timeout-/Retry-Settings
OLLAMA_REQUEST_TIMEOUT_SEC = 220        # Maximale Wartezeit pro Request
OLLAMA_MAX_RETRIES = 4                   # Anzahl Wiederholungen bei Timeout/Verbindungsfehlern
OLLAMA_RETRY_BACKOFF_BASE_SEC = 2        # Basis für exponentielles Backoff (2,4,8...)
OLLAMA_ABORT_ON_TIMEOUT = True           # Bei anhaltender Unerreichbarkeit sauber abbrechen

# Optionales Fallback auf kleineres Modell
OLLAMA_ENABLE_FALLBACK = True
OLLAMA_FALLBACK_MODEL = "llama3.2"

# Test Ollama-Verbindung (nur primäres Modell, keine Fallback-Logik)
try:
    test_response = requests.post(
        OLLAMA_API,
        json={"model": OLLAMA_MODEL, "prompt": "ping", "stream": False, "options": {"num_predict": 4}},
        timeout=30
    )
    if test_response.status_code == 200:
        print(f"✅ Ollama verbunden: {OLLAMA_MODEL}")
        print(f"   Antwort: {test_response.json().get('response','(leer)')}")
        # Testfrage: Funktioniert Reasoning?
        test_prompt = "Du bist Bibliothekar. VDEH: 'Eisen und Stahl' von 'Müller, Hans' (2020). DNB: 'Eisen und Stahl: Geschichte' von 'Müller, H.' (2020). Ist das dasselbe Werk? Antworte: JA oder NEIN."
        test_reason = requests.post(
            OLLAMA_API,
            json={"model": OLLAMA_MODEL, "prompt": test_prompt, "stream": False, "options": {"num_predict": 16}},
            timeout=60
        )
        if test_reason.status_code == 200:
            print(f"   Testfrage: {test_reason.json().get('response','(leer)')}")
        else:
            print(f"⚠️  Testfrage fehlgeschlagen (Status: {test_reason.status_code})")
    else:
        print(f"⚠️  Ollama antwortet nicht korrekt (Status: {test_response.status_code})")
except Exception as e:
    print(f"❌ Ollama nicht erreichbar: {e}")
    print(f"   Stellen Sie sicher, dass Ollama läuft: ollama serve")
    raise

print(f"⚙️  Timeout: {OLLAMA_REQUEST_TIMEOUT_SEC}s | Retries: {OLLAMA_MAX_RETRIES} | Abort: {OLLAMA_ABORT_ON_TIMEOUT}")
print(f"🤖 Aktives Modell: {OLLAMA_MODEL}\n")

📋 === FUSION-SETUP ===

✅ Ollama verbunden: llama3.3:70b
   Antwort: pong!
✅ Ollama verbunden: llama3.3:70b
   Antwort: pong!
   Testfrage: NEIN.
⚙️  Timeout: 220s | Retries: 4 | Abort: True
🤖 Aktives Modell: llama3.3:70b

   Testfrage: NEIN.
⚙️  Timeout: 220s | Retries: 4 | Abort: True
🤖 Aktives Modell: llama3.3:70b



In [4]:
# 🔧 FUSION FUNKTIONEN (erweitert für zwei DNB-Varianten)
print("🔧 === FUSION FUNKTIONEN (Dual-DNB) ===\n")

DNB_VARIANT_PRIORITY = ["id", "title_author"]  # Reihenfolge für Auswahl bei Plausibilität

class OllamaUnavailableError(Exception):
    """Wird ausgelöst, wenn Ollama wiederholt nicht erreichbar ist (Timeout/Verbindungsfehler)."""
    pass

def query_ollama(prompt, model=None, max_retries=None, timeout_sec=None, abort_on_timeout=None):
    """Fragt Ollama-Modell ab mit Retry- und Backoff-Logik.

    Parameter können zur Laufzeit überschrieben werden; ansonsten gelten globale Defaults:
    - model:          OLLAMA_MODEL (ggf. Fallback)
    - max_retries:    OLLAMA_MAX_RETRIES
    - timeout_sec:    OLLAMA_REQUEST_TIMEOUT_SEC
    - abort_on_timeout: OLLAMA_ABORT_ON_TIMEOUT

    Bei wiederholten Timeouts/Verbindungsfehlern wird optional abgebrochen (Exception),
    damit der Aufrufende den Lauf sauber beenden und einen Zwischenstand speichern kann.
    """
    if model is None:
        model = OLLAMA_MODEL
    if max_retries is None:
        max_retries = OLLAMA_MAX_RETRIES
    if timeout_sec is None:
        timeout_sec = OLLAMA_REQUEST_TIMEOUT_SEC
    if abort_on_timeout is None:
        abort_on_timeout = OLLAMA_ABORT_ON_TIMEOUT

    last_err = None
    for attempt in range(max_retries):
        try:
            response = requests.post(
                OLLAMA_API,
                json={
                    "model": model,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": 0.1,
                        "num_predict": 180  # etwas mehr Tokens für zwei Varianten
                    }
                },
                timeout=timeout_sec  # konfigurierbarer Timeout für großes Modell
            )
            if response.status_code == 200:
                return response.json().get('response', '').strip()
            else:
                last_err = RuntimeError(f"HTTP {response.status_code}")
                print(f"   ⚠️ Ollama Fehler (Versuch {attempt+1}/{max_retries}): Status {response.status_code}")
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
            last_err = e
            err_type = 'Timeout' if isinstance(e, requests.exceptions.Timeout) else 'Verbindungsfehler'
            print(f"   ⚠️ Ollama {err_type} (Versuch {attempt+1}/{max_retries}) – erneut versuchen...")
        except Exception as e:
            last_err = e
            print(f"   ⚠️ Unerwarteter Ollama-Fehler (Versuch {attempt+1}/{max_retries}): {e}")
        # Exponentielles Backoff (2,4,8,... capped auf 15s)
        wait_sec = min(OLLAMA_RETRY_BACKOFF_BASE_SEC * (2 ** attempt), 15)
        time.sleep(wait_sec)
    # Nach max_retries: je nach Einstellung entweder Exception werfen (Abbruch) oder None zurückgeben
    if abort_on_timeout:
        raise OllamaUnavailableError(f"Ollama nicht erreichbar nach {max_retries} Versuchen: {last_err}")
    return None

def build_ai_prompt(vdeh, dnb_id, dnb_ta):
    """Erstellt ein Prompt um beide DNB-Varianten gegen den VDEH-Datensatz zu plausibilisieren."""
    def fmt(entry):
        # Defensive: leeres dict falls None
        if entry is None:
            entry = {'title': None, 'authors': None, 'year': None, 'publisher': None}
        return f"- Titel: {entry['title'] if pd.notna(entry['title']) else 'nicht vorhanden'}\n" \
               f"- Autoren: {entry['authors'] if pd.notna(entry['authors']) else 'nicht vorhanden'}\n" \
               f"- Jahr: {entry['year'] if pd.notna(entry['year']) else 'nicht vorhanden'}\n" \
               f"- Verlag: {entry['publisher'] if pd.notna(entry['publisher']) else 'nicht vorhanden'}"
    
    return f"""Du bist ein erfahrener Bibliothekar. Prüfe welche DNB-Variante am besten zu VDEH passt oder ob keine passt.

REGELN:
1. ENTSCHEIDUNGSKRITERIEN: Titel + Autoren dominieren. Jahr ±2 oder fehlend ist OK. Verlag tolerant.
2. SCHREIBWEISEN: Ignoriere Groß-/Kleinschreibung, geringfügige Varianten, Abkürzungen.
3. WENN BEIDE passen: bevorzuge ID-basierte Variante (ISBN/ISSN) gegenüber Titel/Autor.
4. WENN NUR EINE passt: wähle diese.
5. WENN KEINE passt: entscheide NEIN.
6. EIN 'NEIN' nur bei klar unterschiedlichen Werken (Titel UND Autoren deutlich verschieden).
7. Fehlende Felder alleine NIE als Ablehnungsgrund.

DATENSATZ VDEH:\n{fmt(vdeh)}

DNB-VARIANTE A (ID-basiert):\n{fmt(dnb_id)}

DNB-VARIANTE B (Titel/Autor-basiert):\n{fmt(dnb_ta)}

Antworte NUR mit einem dieser Formate:
A - [Begründung]
B - [Begründung]
KEINE - [Begründung warum keine passt]
A&B - [Begründung warum beide gleich gut sind, ID bevorzugt]"""

def parse_ai_choice(response):
    """Parst die KI-Auswahl (A, B, KEINE, A&B)."""
    if not response:
        return 'KEINE', 'KI keine Antwort'
    r = response.strip().upper()
    if r.startswith('A&B'):
        reason = response.split('-',1)[1].strip() if '-' in response else ''
        return 'A', f"Beide passend, ID bevorzugt. {reason}"
    if r.startswith('A ' ) or r.startswith('A-') or r.startswith('A\n') or r == 'A':
        reason = response.split('-',1)[1].strip() if '-' in response else ''
        return 'A', reason
    if r.startswith('B ' ) or r.startswith('B-') or r.startswith('B\n') or r == 'B':
        reason = response.split('-',1)[1].strip() if '-' in response else ''
        return 'B', reason
    if r.startswith('KEINE') or r.startswith('KEIN'):
        reason = response.split('-',1)[1].strip() if '-' in response else ''
        return 'KEINE', reason if reason else 'Keine Variante passt'
    # Fallback: unklare Antwort => KEINE
    return 'KEINE', f"Unklare KI-Antwort: {response}"

def normalize_string(val):
    """Normalisiert Strings für robusteren Vergleich."""
    if pd.isna(val):
        return None
    s = str(val).strip()
    import re, unicodedata
    s = re.sub(r'\s+', ' ', s)
    s = unicodedata.normalize('NFKC', s)
    return s.lower()

def compare_fields(base, other):
    """Vergleicht Felder und erzeugt Konflikt/Bestätigung-Maps."""
    conflicts = {}
    confirmations = {}
    for field in ['title','authors','year','publisher']:
        b = base.get(field)
        o = other.get(field) if other else None
        if pd.notna(b) and pd.notna(o):
            bn = normalize_string(b)
            on = normalize_string(o)
            if bn == on:
                confirmations[field] = str(b)
            else:
                conflicts[field] = {'vdeh': str(b), 'dnb': str(o)}
    return conflicts, confirmations

def merge_record(row):
    """Fusioniert einen Record unter Nutzung beider möglicher DNB-Varianten und KI-Plausibilität."""
    vdeh_data = {
        'title': row.get('title'),
        'authors': row.get('authors_str'),
        'year': row.get('year'),
        'publisher': row.get('publisher')
    }
    # Extrahiere ID-basierte DNB (vorhandene Spalten)
    dnb_id = {
        'title': row.get('dnb_title'),
        'authors': row.get('dnb_authors'),
        'year': row.get('dnb_year'),
        'publisher': row.get('dnb_publisher')
    }
    # Extrahiere Titel/Autor-Variante
    dnb_ta = {
        'title': row.get('dnb_title_ta'),
        'authors': row.get('dnb_authors_ta'),
        'year': row.get('dnb_year_ta'),
        'publisher': row.get('dnb_publisher_ta')
    }
    # Falls komplette Variante fehlt -> setze dict auf None für klarere Prüfung
    id_available = any(pd.notna(dnb_id[f]) for f in dnb_id)
    ta_available = any(pd.notna(dnb_ta[f]) for f in dnb_ta)
    if not id_available:
        dnb_id = None
    if not ta_available:
        dnb_ta = None
    if dnb_id is None and dnb_ta is None:
        return {
            'title': vdeh_data['title'],
            'authors': vdeh_data['authors'],
            'year': vdeh_data['year'],
            'publisher': vdeh_data['publisher'],
            'title_source': 'vdeh',
            'authors_source': 'vdeh',
            'year_source': 'vdeh',
            'publisher_source': 'vdeh',
            'conflicts': None,
            'confirmations': None,
            'ai_reasoning': None,
            'dnb_variant_selected': None,
            'dnb_match_rejected': False,
            'rejection_reason': None
        }
    # KI-Plausibilität
    ai_response = query_ollama(build_ai_prompt(vdeh_data, dnb_id, dnb_ta))
    choice, reason = parse_ai_choice(ai_response)
    if choice == 'KEINE':
        return {
            'title': vdeh_data['title'],
            'authors': vdeh_data['authors'],
            'year': vdeh_data['year'],
            'publisher': vdeh_data['publisher'],
            'title_source': 'vdeh',
            'authors_source': 'vdeh',
            'year_source': 'vdeh',
            'publisher_source': 'vdeh',
            'conflicts': None,
            'confirmations': None,
            'ai_reasoning': f"KI: {reason}",
            'dnb_variant_selected': None,
            'dnb_match_rejected': True,
            'rejection_reason': reason
        }
    selected_variant = 'id' if choice == 'A' else 'title_author'
    selected_data = dnb_id if selected_variant == 'id' else dnb_ta
    conflicts, confirmations = compare_fields(vdeh_data, selected_data)
    result = {
        'conflicts': json.dumps(conflicts, ensure_ascii=False) if conflicts else None,
        'confirmations': json.dumps(confirmations, ensure_ascii=False) if confirmations else None,
        'ai_reasoning': f"KI Entscheidung: Variante {selected_variant} gewählt. {reason}",
        'dnb_match_rejected': False,
        'rejection_reason': None,
        'dnb_variant_selected': selected_variant
    }
    for field in ['title','authors','year','publisher']:
        v_val = vdeh_data[field]
        d_val = selected_data.get(field) if selected_data else None
        if pd.notna(d_val):
            result[field] = d_val
            result[f'{field}_source'] = 'confirmed' if field in confirmations else f'dnb_{selected_variant}'
        elif pd.notna(v_val):
            result[field] = v_val
            result[f'{field}_source'] = 'vdeh'
        else:
            result[field] = None
            result[f'{field}_source'] = None
    return result

print("✅ Funktionen definiert:")
print("   - query_ollama(): konfigurierbarer Timeout + exponentielles Backoff")
print("   - merge_record(): Dual-DNB Varianten mit KI-Auswahl (defensive Null-Handling)")

🔧 === FUSION FUNKTIONEN (Dual-DNB) ===

✅ Funktionen definiert:
   - query_ollama(): konfigurierbarer Timeout + exponentielles Backoff
   - merge_record(): Dual-DNB Varianten mit KI-Auswahl (defensive Null-Handling)


In [ ]:
# 🚀 FUSION AUSFÜHREN
print("🚀 === FUSION AUSFÜHREN ===\n")

# Soll die Fusion komplett neu gestartet werden?
RESET_FUSION = False  # Auf True setzen, um Fortschritt & Ergebnisse zu löschen

# Optional: nur eine kleine Stichprobe verarbeiten (für schnellen Test)
FUSION_LIMIT = None  # Keine Beschränkung mehr
try:
    FUSION_LIMIT = int(config.get('debug.fusion_limit', 0))
    if FUSION_LIMIT <= 0:
        FUSION_LIMIT = None
except Exception:
    FUSION_LIMIT = None

# Statistik VOR Fusion
print("📊 Vollständigkeit VOR Fusion:")
before_stats = {
    'title': df_enriched['title'].notna().sum(),
    'authors': (df_enriched['authors_str'].notna() & (df_enriched['authors_str'] != '')).sum(),
    'year': df_enriched['year'].notna().sum(),
    'publisher': df_enriched['publisher'].notna().sum()
}
for field, count in before_stats.items():
    print(f"   {field}: {count:,} ({count/len(df_enriched)*100:.1f}%)")

# Nur Records mit irgendeiner DNB-Variante verarbeiten (ID oder TA)
has_id = df_enriched[['dnb_title','dnb_authors','dnb_year','dnb_publisher']].notna().any(axis=1) if 'dnb_title' in df_enriched.columns else False
has_ta = df_enriched[['dnb_title_ta','dnb_authors_ta','dnb_year_ta','dnb_publisher_ta']].notna().any(axis=1) if 'dnb_title_ta' in df_enriched.columns else False
records_to_process = df_enriched[has_id | has_ta].copy()
print(f"\n🔄 Verarbeite {len(records_to_process):,} Records mit DNB-Varianten...")
print(f"   (Records ohne DNB-Daten behalten VDEH-Werte)\n")

# 💾 INKREMENTELLES SPEICHERN - Setup
SAVE_INTERVAL = 50  # Speichere alle 50 Records
progress_file = processed_dir / '05_fused_data_progress.parquet'
retry_queue_file = processed_dir / '05_fused_retry_queue.json'

# Fortschritt und Fusionsspalten nur zurücksetzen, wenn RESET_FUSION = True
if RESET_FUSION:
    # Alle Fusionsresultate im DataFrame zurücksetzen
    fusion_cols = [
        'title', 'authors_str', 'year', 'publisher',
        'fusion_title_source', 'fusion_authors_source', 'fusion_year_source', 'fusion_publisher_source',
        'fusion_conflicts', 'fusion_confirmations', 'fusion_ai_reasoning',
        'fusion_dnb_match_rejected', 'fusion_rejection_reason', 'fusion_dnb_variant_selected',
        'fusion_needs_retry', 'fusion_decision_needed'
        # ggf. weitere Fusionsspalten ergänzen
     ]
    for col in fusion_cols:
        if col in df_enriched.columns:
            df_enriched[col] = None
    print("🗑️ Alle Fusionsresultate im DataFrame wurden gelöscht.")

    # Sicherungsdatei jetzt löschen
    import os
    if progress_file.exists():
        try:
            os.remove(progress_file)
            print(f"🗑️ Fortschrittsdatei {progress_file} wurde gelöscht. Starte mit frischer Fusion.")
        except Exception as e:
            print(f"⚠️ Fortschrittsdatei konnte nicht gelöscht werden: {e}")
    else:
        print(f"Keine Fortschrittsdatei gefunden, starte frische Fusion.")

# Prüfe ob bereits verarbeitete Records existieren
already_fused = set()
if progress_file.exists():
    df_progress = pd.read_parquet(progress_file)
    # Robust: falls Index nicht eindeutig → deduplizieren und letzten Stand pro Index behalten
    if not df_progress.index.is_unique:
        df_progress = df_progress[~df_progress.index.duplicated(keep='last')]
    # Defensive: nur fortfahren, wenn die Spalte existiert
    if 'fusion_title_source' in df_progress.columns:
        already_fused = set(df_progress[df_progress['fusion_title_source'].notna()].index)
    else:
        already_fused = set()
    print(f"📂 Fortschritt geladen: {len(already_fused):,} Records bereits fusioniert")

    # Übernehme bereits fusionierte Daten (bulk, spaltenweise)
    common_cols = [c for c in df_progress.columns if c in df_enriched.columns]
    if len(common_cols) > 0 and len(already_fused) > 0:
        idxs = [i for i in already_fused if i in df_enriched.index]
        if len(idxs) > 0:
            # Align auf gemeinsamen Index
            aligned_progress = df_progress.loc[idxs, common_cols]
            df_enriched.loc[idxs, common_cols] = aligned_progress.values
        print(f"   Fusionsdaten wiederhergestellt\n")

# Filtere bereits verarbeitete Records
records_to_process = records_to_process[~records_to_process.index.isin(already_fused)]

# Optional limit (Testmodus)
if FUSION_LIMIT and FUSION_LIMIT > 0:
    print(f"🧪 Testmodus aktiv – verarbeite nur die ersten {FUSION_LIMIT} Records.")
    records_to_process = records_to_process.head(FUSION_LIMIT)

# 🔁 Retry-Queue laden und priorisieren
retry_indices = []
if retry_queue_file.exists():
    try:
        with open(retry_queue_file, 'r', encoding='utf-8') as f:
            retry_indices = json.load(f)
            if not isinstance(retry_indices, list):
                retry_indices = []
    except Exception:
        retry_indices = []

# Nur solche Indizes berücksichtigen, die wirklich noch zu verarbeiten sind
retry_indices = [i for i in retry_indices if i in records_to_process.index]

if len(retry_indices) > 0:
    print(f"🔁 Retry-Queue erkannt: {len(retry_indices):,} Records werden zuerst verarbeitet")
    retry_df = records_to_process.loc[records_to_process.index.isin(retry_indices)]
    fresh_df = records_to_process.loc[~records_to_process.index.isin(retry_indices)]
    records_to_process = pd.concat([retry_df, fresh_df], axis=0)

print(f"🔄 Verbleibende Records: {len(records_to_process):,}\n")

# Statistiken
fusion_stats = {
    'total_processed': len(already_fused),
    'conflicts_found': 0,
    'dnb_preferred': 0,  # DNB gewählt (mit Unterschied)
    'simple_merges': 0,
    'errors': 0,
    'dnb_matches_rejected': 0,
    'ai_decisions': 0,
    'variant_id': 0,
    'variant_title_author': 0,
    'variant_none': 0
}

from tqdm.auto import tqdm

fusion_count = 0  # Counter für inkrementelles Speichern
aborted = False  # Abbruch-Flag bei Ollama-Timeout/Verbindungsfehler

for idx, row in tqdm(records_to_process.iterrows(), total=len(records_to_process), desc="🔄 Fusion", unit="records"):
    # === BEGIN: DNB VARIANT USAGE PRINTOUT ===
    has_id_variant = any([pd.notna(row.get(col)) for col in ['dnb_title','dnb_authors','dnb_year','dnb_publisher']]) if 'dnb_title' in row else False
    has_ta_variant = any([pd.notna(row.get(col)) for col in ['dnb_title_ta','dnb_authors_ta','dnb_year_ta','dnb_publisher_ta']]) if 'dnb_title_ta' in row else False
    variant_str = []
    if has_id_variant:
        variant_str.append("ISBN/ID-basierte DNB-Variante")
    if has_ta_variant:
        variant_str.append("Titel/Autor-basierte DNB-Variante")
    if not variant_str:
        variant_str.append("Keine DNB-Variante")
    print(f"\n📚 Record {idx}: {' + '.join(variant_str)}")
    # === END: DNB VARIANT USAGE PRINTOUT ===
    # === BEGIN: TITEL/AUTHOR VERGLEICH AUSGABE ===
    if has_id_variant or has_ta_variant:
        print("   --- Vergleich der Varianten (Titel/Autor) ---")
        vdeh_title = (str(row.get('title', 'N/A')) or 'N/A')[:60]
        vdeh_authors = (str(row.get('authors_str', 'N/A')) or 'N/A')[:60]
        dnb_id_title = (str(row.get('dnb_title', 'N/A')) or 'N/A')[:60]
        dnb_id_authors = (str(row.get('dnb_authors', 'N/A')) or 'N/A')[:60]
        dnb_ta_title = (str(row.get('dnb_title_ta', 'N/A')) or 'N/A')[:60]
        dnb_ta_authors = (str(row.get('dnb_authors_ta', 'N/A')) or 'N/A')[:60]
        print(f"   VDEH:   {vdeh_title} | {vdeh_authors}")
        print(f"   DNB ID: {dnb_id_title} | {dnb_id_authors}")
        print(f"   DNB TA: {dnb_ta_title} | {dnb_ta_authors}")
        print("   ------------------------------------------")
    # === END: TITEL/AUTHOR VERGLEICH AUSGABE ===
    # === BEGIN: MARKIERUNG REDUNDANZ/ENTSCHEIDUNG ===
    # Markiere, ob eine Entscheidung nötig war (Konflikt oder KI-Entscheidung)
    decision_needed = False
    # Wird nach merge_record gesetzt, siehe unten
    # === END: MARKIERUNG REDUNDANZ/ENTSCHEIDUNG ===
    try:
        # Fusion durchführen
        fusion_result = merge_record(row)
        
        # Entscheidung nötig, wenn Konflikte oder KI-Entscheidung
        decision_needed = bool(fusion_result.get('conflicts')) or bool(fusion_result.get('ai_reasoning'))
        df_enriched.loc[idx, 'fusion_decision_needed'] = decision_needed

        # Statistik: Variante
        variant = fusion_result.get('dnb_variant_selected')
        if variant == 'id':
            fusion_stats['variant_id'] += 1
        elif variant == 'title_author':
            fusion_stats['variant_title_author'] += 1
        else:
            fusion_stats['variant_none'] += 1
        
        # 🚫 VERWORFENE DNB-MATCHES ANZEIGEN
        if fusion_result.get('dnb_match_rejected'):
            print(f"\n   {'='*70}")
            print(f"   🚫 DNB-MATCH VERWORFEN bei Record {idx}")
            print(f"   {'='*70}")
            print(f"\n   Grund: {fusion_result.get('rejection_reason')}")
            print(f"\n   VDEH: {vdeh_title}...")
            print(f"   DNB ID:  {dnb_id_title}...")
            print(f"   DNB TA:  {dnb_ta_title}...")
            print(f"\n   → Nur VDEH-Daten verwendet")
            print(f"\n   {'='*70}\n")
        
        # 🔍 ECHTZEIT-KONFLIKT-ANZEIGE
        if fusion_result.get('conflicts'):
            try:
                conflicts = json.loads(fusion_result.get('conflicts'))
                
                print(f"\n   {'='*70}")
                print(f"   📊 UNTERSCHIEDE bei Record {idx} (DNB autoritativ)")
                print(f"   {'='*70}")
                
                # Zeige konkrete Werte für jedes Konfliktfeld
                for field, values in conflicts.items():
                    vdeh_val = values.get('vdeh', 'N/A')
                    dnb_val = values.get('dnb', 'N/A')
                    source = fusion_result.get(f'{field}_source', 'N/A').upper()
                    
                    print(f"\n   📌 {field.upper()}:")
                    print(f"      VDEH: {vdeh_val}")
                    print(f"      DNB:  {dnb_val}")
                    print(f"      ✅ Gewählt: {source}")
                
                print(f"\n   {'='*70}\n")
                
            except Exception as e:
                print(f"\n   ⚠️  Konflikt bei Record {idx} (Fehler beim Parsen: {e})\n")
        
        # Ergebnisse in DataFrame speichern
        df_enriched.loc[idx, 'title'] = fusion_result.get('title')
        df_enriched.loc[idx, 'authors_str'] = fusion_result.get('authors')
        
        # ⚠️ WICHTIG: year als Int64 konvertieren (kann String sein von KI)
        year_val = fusion_result.get('year')
        if pd.notna(year_val):
            try:
                df_enriched.loc[idx, 'year'] = pd.to_numeric(year_val, errors='coerce')
            except:
                df_enriched.loc[idx, 'year'] = year_val
        else:
            df_enriched.loc[idx, 'year'] = None
        
        df_enriched.loc[idx, 'publisher'] = fusion_result.get('publisher')
        
        df_enriched.loc[idx, 'fusion_title_source'] = fusion_result.get('title_source')
        df_enriched.loc[idx, 'fusion_authors_source'] = fusion_result.get('authors_source')
        df_enriched.loc[idx, 'fusion_year_source'] = fusion_result.get('year_source')
        df_enriched.loc[idx, 'fusion_publisher_source'] = fusion_result.get('publisher_source')
        
        df_enriched.loc[idx, 'fusion_conflicts'] = fusion_result.get('conflicts')
        df_enriched.loc[idx, 'fusion_confirmations'] = fusion_result.get('confirmations')
        df_enriched.loc[idx, 'fusion_ai_reasoning'] = fusion_result.get('ai_reasoning')
        df_enriched.loc[idx, 'fusion_dnb_match_rejected'] = fusion_result.get('dnb_match_rejected', False)
        df_enriched.loc[idx, 'fusion_rejection_reason'] = fusion_result.get('rejection_reason')
        df_enriched.loc[idx, 'fusion_dnb_variant_selected'] = fusion_result.get('dnb_variant_selected')
        
        # Falls dieser Record zuvor als Retry markiert war -> zurücksetzen und aus Queue entfernen
        if 'fusion_needs_retry' in df_enriched.columns and bool(df_enriched.loc[idx, 'fusion_needs_retry']):
            df_enriched.loc[idx, 'fusion_needs_retry'] = False
        if idx in retry_indices:
            retry_indices = [i for i in retry_indices if i != idx]
        
        # Statistiken
        fusion_stats['total_processed'] += 1
        fusion_count += 1
        fusion_stats['ai_decisions'] += 1
        
        if fusion_result.get('dnb_match_rejected'):
            fusion_stats['dnb_matches_rejected'] += 1
        elif fusion_result.get('conflicts'):
            fusion_stats['conflicts_found'] += 1
            fusion_stats['dnb_preferred'] += 1  # DNB gewählt
        else:
            fusion_stats['simple_merges'] += 1
        
        # 💾 INKREMENTELLES SPEICHERN
        if fusion_count % SAVE_INTERVAL == 0:
            df_enriched.to_parquet(progress_file, index=True)
            # Retry-Queue persistieren
            try:
                with open(retry_queue_file, 'w', encoding='utf-8') as f:
                    json.dump(retry_indices, f, ensure_ascii=False, indent=2)
            except Exception:
                pass
            print(f"\n   💾 Zwischenstand gespeichert: {fusion_stats['total_processed']:,} Records fusioniert")
        
    except OllamaUnavailableError as e:
        # Verbindungsabbruch/Timeout -> Record für Retry markieren, speichern und ggf. Lauf abbrechen
        print(f"\n❌ Ollama nicht erreichbar: {e}")
        print("👉 Record wird in die Retry-Queue gelegt und der Lauf sauber beendet.")
        fusion_stats['errors'] += 1
        fusion_stats['aborted'] = True
        aborted = True
        # aktuellen Record markieren
        df_enriched.loc[idx, 'fusion_needs_retry'] = True
        if idx not in retry_indices:
            retry_indices.append(idx)
        # Sofort Zwischenstand sichern
        df_enriched.to_parquet(progress_file, index=True)
        try:
            with open(retry_queue_file, 'w', encoding='utf-8') as f:
                json.dump(retry_indices, f, ensure_ascii=False, indent=2)
        except Exception:
            pass
        print(f"\n💾 Zwischenstand + Retry-Queue gespeichert (Records in Queue: {len(retry_indices):,})")
        break
    except Exception as e:
        print(f"\n   ⚠️ Fehler bei Record {idx}: {e}")
        fusion_stats['errors'] += 1

# 💾 FINALES SPEICHERN (falls letzte Gruppe < SAVE_INTERVAL)
if fusion_count % SAVE_INTERVAL != 0 or fusion_count == 0:
    df_enriched.to_parquet(progress_file, index=True)
    # Retry-Queue persistieren (letzter Stand)
    try:
        with open(retry_queue_file, 'w', encoding='utf-8') as f:
            json.dump(retry_indices, f, ensure_ascii=False, indent=2)
    except Exception:
        pass
    print(f"\n💾 Finaler Zwischenstand gespeichert")

if aborted:
    print("\n⛔️ Lauf wurde aufgrund von Ollama-Timeout/Verbindungsfehler abgebrochen.\n   Nach Wiederherstellung der Verbindung startet der nächste Lauf automatisch mit der Retry-Queue.")

# Statistik NACH Fusion
print(f"\n📊 Vollständigkeit NACH Fusion:")
after_stats = {
    'title': df_enriched['title'].notna().sum(),
    'authors': (df_enriched['authors_str'].notna() & (df_enriched['authors_str'] != '')).sum(),
    'year': df_enriched['year'].notna().sum(),
    'publisher': df_enriched['publisher'].notna().sum()
}
for field, count in after_stats.items():
    improvement = count - before_stats[field]
    print(f"   {field}: {count:,} ({count/len(df_enriched)*100:.1f}%) [+{improvement:,}]")

# Fusion-Statistiken
print(f"\n📊 === FUSION-STATISTIKEN ===")
print(f"   Verarbeitet: {fusion_stats['total_processed']:,}")
print(f"   Einfache Merges: {fusion_stats['simple_merges']:,}")
print(f"   DNB gewählt (Unterschiede): {fusion_stats['dnb_preferred']:,}")
print(f"   Unterschiede gefunden: {fusion_stats['conflicts_found']:,}")
print(f"   🚫 DNB-Matches verworfen: {fusion_stats['dnb_matches_rejected']:,}")
print(f"   KI-Entscheidungen: {fusion_stats['ai_decisions']:,}")
print(f"   Variante ID: {fusion_stats['variant_id']:,}")
print(f"   Variante Titel/Autor: {fusion_stats['variant_title_author']:,}")
print(f"   Variante keine: {fusion_stats['variant_none']:,}")
if aborted:
    print(f"   ⛔️ Abgebrochen: Ja (Ollama nicht erreichbar)")

# Quellen-Verteilung
print(f"\n📊 Datenquellen nach Fusion:")
for field in ['title', 'authors', 'year', 'publisher']:
    source_col = f'fusion_{field}_source'
    if source_col in df_enriched.columns:
        sources = df_enriched[source_col].value_counts()
        print(f"\n   {field.upper()}:")
        for source, count in sources.items():
            if source:
                print(f"     {source}: {count:,}")

print(f"\n✅ Fusion abgeschlossen")

🚀 === FUSION AUSFÜHREN ===

📊 Vollständigkeit VOR Fusion:
   title: 20 (0.0%)
   authors: 19 (0.0%)
   year: 20 (0.0%)
   publisher: 7 (0.0%)

🔄 Verarbeite 9,111 Records mit DNB-Varianten...
   (Records ohne DNB-Daten behalten VDEH-Werte)

🔄 Verbleibende Records: 9,111



🔄 Fusion:   0%|          | 0/9111 [00:00<?, ?records/s]


📚 Record 16: Titel/Autor-basierte DNB-Variante
   --- Vergleich der Varianten (Titel/Autor) ---
   VDEH:   Sinter als Einsatzmaterial für das Feststoffdirektreduktion | Romberg, Michael
   DNB ID: None | None
   DNB TA: Sinter als Einsatzmaterial für das Feststoffdirektreduktion | Romberg, Michael
   ------------------------------------------

📚 Record 17: Titel/Autor-basierte DNB-Variante
   --- Vergleich der Varianten (Titel/Autor) ---
   VDEH:   Beitrag zum Austenitisierungsprozess der Stähle | Reichelt, Gundolf
   DNB ID: None | None
   DNB TA: Beitrag zum Austenitisierungsprozess der Stähle | Reichelt, Gundolf
   ------------------------------------------

📚 Record 17: Titel/Autor-basierte DNB-Variante
   --- Vergleich der Varianten (Titel/Autor) ---
   VDEH:   Beitrag zum Austenitisierungsprozess der Stähle | Reichelt, Gundolf
   DNB ID: None | None
   DNB TA: Beitrag zum Austenitisierungsprozess der Stähle | Reichelt, Gundolf
   ------------------------------------------
